# Process GREIN Human Data

Retrieve the downloaded expression data, update gene identifiers to entrez, and curate sample IDs. The script will also identify a balanced hold-out test set to compare projection performance into learned latent spaces across algorithms.

In [4]:
import os
import random
import pandas as pd
from sklearn.model_selection import train_test_split

In [5]:
random.seed(1234)

## Read Phenotype Information: skipping until phenotype data found

In [3]:
# path = os.path.join('download', 'TARGET_phenotype.gz')
# pheno_df = pd.read_table(path)

# print(pheno_df.shape)
# pheno_df.head(3)

## Read Probe Mapping Info 
(where chromosomes start and end, saved in a file in downloads)

In [4]:
# path = os.path.join('download', 'gencode.v23.annotation.gene.probemap')
# probe_map_df = pd.read_table(path)

# # Inner merge gene df to get ensembl to entrez mapping
# probe_map_df = probe_map_df.merge(gene_df, how='inner', left_on='gene', right_on='symbol')

# # Mapping to rename gene expression index
# ensembl_to_entrez = dict(zip(probe_map_df.id, probe_map_df.entrez_gene_id))

# print(probe_map_df.shape)
# probe_map_df.head(3)

## Read Entrez ID Curation Info

In [6]:
# Commit from https://github.com/cognoma/genes
genes_commit = 'ec0e5e5fc0066a6c1829866cccab882b7888d709'
url = 'https://raw.githubusercontent.com/cognoma/genes/{}/data/genes.tsv'.format(genes_commit)
gene_df = pd.read_table(url)

# Only consider protein-coding genes
gene_df = (
    gene_df.query("gene_type == 'protein-coding'")
)

gene_df.head()

,entrez_gene_id,symbol,description,chromosome,gene_type,synonyms,aliases
0,1,A1BG,alpha-1-B glycoprotein,19,protein-coding,A1B|ABG|GAB|HYST2477,alpha-1B-glycoprotein|HEL-S-163pA|epididymis s...
1,2,A2M,alpha-2-macroglobulin,12,protein-coding,A2MD|CPAMD5|FWP007|S863-7,alpha-2-macroglobulin|C3 and PZP-like alpha-2-...
3,9,NAT1,N-acetyltransferase 1,8,protein-coding,AAC1|MNAT|NAT-1|NATI,arylamine N-acetyltransferase 1|N-acetyltransf...
4,10,NAT2,N-acetyltransferase 2,8,protein-coding,AAC2|NAT-2|PNAT,arylamine N-acetyltransferase 2|N-acetyltransf...
6,12,SERPINA3,serpin family A member 3,14,protein-coding,AACT|ACT|GIG24|GIG25,alpha-1-antichymotrypsin|cell growth-inhibitin...


In [7]:
# Load gene updater - old to new Entrez gene identifiers
url = 'https://raw.githubusercontent.com/cognoma/genes/{}/data/updater.tsv'.format(genes_commit)
updater_df = pd.read_table(url)
old_to_new_entrez = dict(zip(updater_df.old_entrez_gene_id,
                             updater_df.new_entrez_gene_id))

In [8]:
# Check whether entrez ids are up to date (they are in the version downloaded from cognoma using the commit ID I included)
# However, it's possible this could change in the future
try:
    assert len(set(gene_df["entrez_gene_id"].unique()) & old_to_new_entrez.keys()) == 0
except:
    gene_df = gene_df.replace({"entrez_gene_id": old_to_new_entrez})

In [9]:
# Map gene symbols onto Entrez IDs
assign_entrez_ids = dict(zip(gene_df.symbol,
                             gene_df.entrez_gene_id))

## Read Gene Expression Data

In [10]:
# reading in human gene expression data

file = os.path.join('download', 'grein_count_matrix_human.pkl')
expr_df = pd.read_pickle(file)

print(expr_df.shape)
expr_df.head(4)

(27990, 57)


,gene,gene_symbol,GSM2667747,GSM2667748,GSM2667749,GSM2667750,GSM2667751,GSM2667752,GSM2667753,GSM2667754,...,GSM2668086,GSM2668087,GSM2668088,GSM2668089,GSM2671001,GSM2671002,GSM2671003,GSM2671004,GSM2671005,GSM2671006
0,ENSG00000000003,TSPAN6,1766.0677,420.8206,300.8009,3142.8439,2207.8392,4367.5056,228.5759,650.3287,...,254.7605,143.2263,174.3319,202.3253,1053.8788,2405.3001,940.5770,973.7475,760.0100,1472.2712
1,ENSG00000000005,TNMD,43.9280,18.0091,0.0000,115.8917,63.5936,39.7803,11.7588,1.0153,...,0.0000,0.0000,0.0000,0.0000,3.9978,11.0025,4.9979,8.0054,3.9978,11.0011
2,ENSG00000000419,DPM1,1097.7855,367.2333,316.5226,3895.0588,1536.1375,1084.5276,127.7205,245.1014,...,613.1214,374.0965,302.6622,454.0570,1227.1084,2980.0756,1045.8061,1098.7163,759.7729,1398.8077
3,ENSG00000000457,SCYL3,601.4702,270.9239,163.6869,1177.2241,873.9074,845.1423,229.8548,134.4694,...,196.5995,185.3106,260.9485,183.7584,586.7993,1504.0115,619.3764,661.9514,469.5312,1113.1412


In [11]:
# Clean up x and y from expr_df

# create a list of cleaned columns
cleaned_column = ['gene', 'gene_symbol']
for column in expr_df.drop(['gene', 'gene_symbol'], axis = 1).columns:
    cleaned_column.append(column.split("_")[0])

# set colums to cleaned column
expr_df.columns = cleaned_column

In [67]:
# Check that it worked

pd.set_option('display.max_columns', None) 
expr_df.head()

,gene,gene_symbol,GSM2667747,GSM2667748,GSM2667749,GSM2667750,GSM2667751,GSM2667752,GSM2667753,GSM2667754,GSM2667755,GSM2667756,GSM2667757,GSM2667758,GSM2667759,GSM2667760,GSM2667761,GSM2667762,GSM2667763,GSM2667764,GSM2667765,GSM2667766,GSM2667767,GSM2667768,GSM2667769,GSM2667770,GSM2667771,GSM2667772,GSM2667773,GSM2667774,GSM2667775,GSM2667776,GSM2667777,GSM2667778,GSM2667779,GSM2667780,GSM2667781,GSM2667782,GSM2667783,GSM2667784,GSM2667785,GSM2667786,GSM2668081,GSM2668082,GSM2668083,GSM2668084,GSM2668085,GSM2668086,GSM2668087,GSM2668088,GSM2668089,GSM2671001,GSM2671002,GSM2671003,GSM2671004,GSM2671005,GSM2671006
0,ENSG00000000003,TSPAN6,1766.0677,420.8206,300.8009,3142.8439,2207.8392,4367.5056,228.5759,650.3287,4880.0185,8629.5404,2951.1930,370.2859,321.4122,3721.4916,3499.9075,2462.5672,300.7646,370.7972,3470.2847,3503.2877,3362.5934,226.8240,131.7482,4486.6134,7225.4738,3185.4922,531.7498,796.0883,3175.4423,2064.0788,1267.6230,1845.9171,409.7910,2540.8740,2906.8171,2829.6631,1920.6238,1165.6860,1874.8260,2539.2382,265.0806,300.3062,331.6881,183.7755,278.5022,254.7605,143.2263,174.3319,202.3253,1053.8788,2405.3001,940.5770,973.7475,760.0100,1472.2712
1,ENSG00000000005,TNMD,43.9280,18.0091,0.0000,115.8917,63.5936,39.7803,11.7588,1.0153,67.1228,61.4995,54.1517,11.8111,5.7130,79.6294,79.6162,69.2199,13.8240,5.9650,138.7558,84.4619,181.6344,46.6972,12.8469,168.0059,403.5948,58.7409,18.8835,12.0940,67.2416,10.9099,133.5243,147.7444,15.2011,202.6725,343.3962,676.6289,376.7828,141.2399,380.3029,662.0918,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,3.9978,11.0025,4.9979,8.0054,3.9978,11.0011
2,ENSG00000000419,DPM1,1097.7855,367.2333,316.5226,3895.0588,1536.1375,1084.5276,127.7205,245.1014,2608.6175,3797.5758,855.0836,124.4127,220.1181,2932.3449,1795.7509,1499.5849,264.1154,536.7919,3716.6839,1998.1502,1210.7017,112.5526,150.2928,3655.9428,2458.6382,834.2019,135.4603,258.5075,2264.3384,1241.8778,217.2588,623.3691,165.4709,1166.0064,1281.7182,952.9915,635.3723,879.0727,1695.9609,1736.9615,537.7681,461.4597,436.1302,416.2508,568.8483,613.1214,374.0965,302.6622,454.0570,1227.1084,2980.0756,1045.8061,1098.7163,759.7729,1398.8077
3,ENSG00000000457,SCYL3,601.4702,270.9239,163.6869,1177.2241,873.9074,845.1423,229.8548,134.4694,676.4072,1177.2415,1155.1429,232.3545,115.9401,1458.9760,1182.6409,907.3663,335.9731,153.7599,1083.9457,711.6130,1259.2643,275.6282,30.2989,1144.3081,1374.0298,1118.6175,325.3986,202.9192,1189.7186,881.3931,124.4839,948.1346,57.5744,547.4528,439.9557,1454.0821,1793.4598,232.8164,773.3686,939.7315,157.2023,151.5847,196.0139,142.7542,170.0119,196.5995,185.3106,260.9485,183.7584,586.7993,1504.0115,619.3764,661.9514,469.5312,1113.1412
4,ENSG00000000460,C1orf112,1040.8861,349.1418,298.1800,1967.3975,2115.2425,980.9359,253.4457,145.7038,1024.4449,1294.0961,599.8307,117.7714,67.3678,785.8567,466.6687,1315.1359,300.5638,246.0739,1469.0028,1906.7889,1471.1647,229.3009,67.8212,1176.1864,1307.8388,678.5934,177.1994,92.6689,560.3419,386.5186,92.7886,354.3580,49.4149,211.9013,110.9184,673.9635,749.7606,86.1525,266.8906,338.4995,83.2406,63.1939,50.6896,71.3720,161.8204,204.1681,186.2225,127.6457,172.0572,441.0758,1258.3932,466.5542,450.2848,289.2767,467.7875


## Process gene expression matrix

This involves updating Entrez gene ids, sorting and subsetting

In [70]:
expr_df = (expr_df
           .dropna(axis='rows')
           .set_index("gene_symbol")
           .rename(index=assign_entrez_ids)
           .rename_axis('entrez_gene_id')
     #.groupby(level=0).mean()
     # .transpose()
     # .sort_index(axis='rows')
     # .sort_index(axis='columns')
 )

#expr_df.index.rename('sample_id', inplace=True)

print(expr_df.shape)
expr_df.head()

(27206, 56)


,gene,GSM2667747,GSM2667748,GSM2667749,GSM2667750,GSM2667751,GSM2667752,GSM2667753,GSM2667754,GSM2667755,GSM2667756,GSM2667757,GSM2667758,GSM2667759,GSM2667760,GSM2667761,GSM2667762,GSM2667763,GSM2667764,GSM2667765,GSM2667766,GSM2667767,GSM2667768,GSM2667769,GSM2667770,GSM2667771,GSM2667772,GSM2667773,GSM2667774,GSM2667775,GSM2667776,GSM2667777,GSM2667778,GSM2667779,GSM2667780,GSM2667781,GSM2667782,GSM2667783,GSM2667784,GSM2667785,GSM2667786,GSM2668081,GSM2668082,GSM2668083,GSM2668084,GSM2668085,GSM2668086,GSM2668087,GSM2668088,GSM2668089,GSM2671001,GSM2671002,GSM2671003,GSM2671004,GSM2671005,GSM2671006
entrez_gene_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
7105,ENSG00000000003,1766.0677,420.8206,300.8009,3142.8439,2207.8392,4367.5056,228.5759,650.3287,4880.0185,8629.5404,2951.1930,370.2859,321.4122,3721.4916,3499.9075,2462.5672,300.7646,370.7972,3470.2847,3503.2877,3362.5934,226.8240,131.7482,4486.6134,7225.4738,3185.4922,531.7498,796.0883,3175.4423,2064.0788,1267.6230,1845.9171,409.7910,2540.8740,2906.8171,2829.6631,1920.6238,1165.6860,1874.8260,2539.2382,265.0806,300.3062,331.6881,183.7755,278.5022,254.7605,143.2263,174.3319,202.3253,1053.8788,2405.3001,940.5770,973.7475,760.0100,1472.2712
64102,ENSG00000000005,43.9280,18.0091,0.0000,115.8917,63.5936,39.7803,11.7588,1.0153,67.1228,61.4995,54.1517,11.8111,5.7130,79.6294,79.6162,69.2199,13.8240,5.9650,138.7558,84.4619,181.6344,46.6972,12.8469,168.0059,403.5948,58.7409,18.8835,12.0940,67.2416,10.9099,133.5243,147.7444,15.2011,202.6725,343.3962,676.6289,376.7828,141.2399,380.3029,662.0918,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,3.9978,11.0025,4.9979,8.0054,3.9978,11.0011
8813,ENSG00000000419,1097.7855,367.2333,316.5226,3895.0588,1536.1375,1084.5276,127.7205,245.1014,2608.6175,3797.5758,855.0836,124.4127,220.1181,2932.3449,1795.7509,1499.5849,264.1154,536.7919,3716.6839,1998.1502,1210.7017,112.5526,150.2928,3655.9428,2458.6382,834.2019,135.4603,258.5075,2264.3384,1241.8778,217.2588,623.3691,165.4709,1166.0064,1281.7182,952.9915,635.3723,879.0727,1695.9609,1736.9615,537.7681,461.4597,436.1302,416.2508,568.8483,613.1214,374.0965,302.6622,454.0570,1227.1084,2980.0756,1045.8061,1098.7163,759.7729,1398.8077
57147,ENSG00000000457,601.4702,270.9239,163.6869,1177.2241,873.9074,845.1423,229.8548,134.4694,676.4072,1177.2415,1155.1429,232.3545,115.9401,1458.9760,1182.6409,907.3663,335.9731,153.7599,1083.9457,711.6130,1259.2643,275.6282,30.2989,1144.3081,1374.0298,1118.6175,325.3986,202.9192,1189.7186,881.3931,124.4839,948.1346,57.5744,547.4528,439.9557,1454.0821,1793.4598,232.8164,773.3686,939.7315,157.2023,151.5847,196.0139,142.7542,170.0119,196.5995,185.3106,260.9485,183.7584,586.7993,1504.0115,619.3764,661.9514,469.5312,1113.1412
55732,ENSG00000000460,1040.8861,349.1418,298.1800,1967.3975,2115.2425,980.9359,253.4457,145.7038,1024.4449,1294.0961,599.8307,117.7714,67.3678,785.8567,466.6687,1315.1359,300.5638,246.0739,1469.0028,1906.7889,1471.1647,229.3009,67.8212,1176.1864,1307.8388,678.5934,177.1994,92.6689,560.3419,386.5186,92.7886,354.3580,49.4149,211.9013,110.9184,673.9635,749.7606,86.1525,266.8906,338.4995,83.2406,63.1939,50.6896,71.3720,161.8204,204.1681,186.2225,127.6457,172.0572,441.0758,1258.3932,466.5542,450.2848,289.2767,467.7875


In [71]:
# Make sure that all of the gene symbols were assigned an ID
assert sum(expr_df.index.isnull()) == 0

## Stratify Balanced Training and Testing Sets in TARGET Gene Expression

Output training and testing gene expression datasets

In [9]:
#strat = pheno_df.set_index('sample_id').reindex(expr_df.index).primary_disease_code

In [10]:
# cancertype_count_df = (
#     pd.DataFrame(strat.value_counts()) #not using value_counts, what number should we use here? 
#     .reset_index()
#     .rename({'index': 'cancertype', 'primary_disease_code': 'n ='}, axis='columns')
# )

# file = os.path.join('data', 'target_sample_counts.tsv') #change which file - do we have a file that works for this?
# cancertype_count_df.to_csv(file, sep='\t', index=False)

# cancertype_count_df

In [11]:
train_df, test_df = train_test_split(expr_df,
                                     test_size=0.1,
                                     random_state=123) #if no stratify defined, should just randomize on its own

In [12]:
print(train_df.shape)
test_df.shape

(25191, 1331)


(2799, 1331)

In [13]:
#save train dataframe to file 
train_file = os.path.join('data', 'train_grein_human_expression_matrix_processed.tsv.gz')
train_df.to_csv(train_file, sep='\t', compression='gzip', float_format='%.3g')

In [14]:
#save test dataframe to file 
test_file = os.path.join('data', 'test_grein_human_expression_matrix_processed.tsv.gz')
test_df.to_csv(test_file, sep='\t', compression='gzip', float_format='%.3g')

## Sort genes based on median absolute deviation and output to file

In [27]:
# function to calculate median absolute deviation
def mad(x): 
    abs(x - x.median(axis='columns', numeric_only=True)).median(axis='columns', numeric_only=True)

In [17]:
# add gene_id as a column in dataframe
train_df['gene_id'] = train_df['gene'] + train_df['gene_symbol']
col = train_df.pop('gene_id')
train_df.insert(0, col.name, col)

train_df.head()

gene_id             gene gene_symbol  GSM2667747  \
sample_id                                                                    
27744         ENSG00000283046HTN1  ENSG00000283046        HTN1    125.4665   
9537         ENSG00000153037SRP19  ENSG00000153037       SRP19    894.5558   
9769        ENSG00000155115GTF3C6  ENSG00000155115      GTF3C6    851.6143   
15239      ENSG00000184083FAM120C  ENSG00000184083     FAM120C    632.3881   
203         ENSG00000008196TFAP2B  ENSG00000008196      TFAP2B     23.2873   

           GSM2667748  GSM2667749  GSM2667750  GSM2667751  GSM2667752  \
sample_id                                                               
27744        140.2465     17.6274     69.7745     73.7733     27.6203   
9537         357.1081    187.4946   1421.0479    456.0143    802.6670   
9769         187.0203    243.4404   2046.5828    476.0532   1977.8905   
15239        285.1058    122.8606    832.8702    957.4550   1495.5920   
203            2.0018     19.1983     54.7994     19.0940    974.5680   

           GSM2667753  GSM2667754  GSM2667755  GSM2667756  GSM2667757  \
sample_id                                                               
27744         98.1395     12.7103      3.6260     24.9714     51.9728   
9537         124.8560    164.7368   3532.8830   1244.8740   1010.8748   
9769          94.7822    881.8821  10161.3111   1382.6290   1326.2597   
15239        184.7934     91.7322    383.1359   1374.5838   1628.9039   
203           88.7088     88.8378    620.9386   1124.1208   6430.8975   

           GSM2667758  GSM2667759  GSM2667760  GSM2667761  GSM2667762  \
sample_id                                                               
27744         66.2000      3.0588     29.6436     78.6089    192.3800   
9537         139.9850    138.3330   3899.5298    930.8268   1137.9553   
9769         141.7311    477.2282   5214.7134    767.6003    982.4279   
15239        232.1640    120.3054    898.8325   1279.4379    824.3853   
203          744.3863    499.9203   5383.4356   7004.7640     18.7766   

           GSM2667763  GSM2667764  GSM2667765  GSM2667766  GSM2667767  \
sample_id                                                               
27744        153.2949      4.0605     61.9086    132.9720    178.1983   
9537         254.7177    245.6310   1621.9018    456.3314   1035.1774   
9769         117.6392    369.8380   2667.1936    459.5399   1652.4284   
15239        188.6935    166.3629    881.8608   1010.1709   1574.6568   
203            6.8329      7.8788     36.7517     19.2128   1565.6738   

           GSM2667768  GSM2667769  GSM2667770  GSM2667771  GSM2667772  \
sample_id                                                               
27744        117.1625      6.5244     18.5502     57.2569    106.8320   
9537         116.1225    123.1838   2368.7324    921.9137   1162.6473   
9769          63.1625    275.7449   3911.4982   1060.2774   1230.0665   
15239        261.7398     58.3872    756.5876   2166.1719   1840.3923   
203          132.5760     22.7705   1405.0990   2319.8399   7436.2614   

           GSM2667773  GSM2667774  GSM2667775  GSM2667776  GSM2667777  \
sample_id                                                               
27744        148.3160     14.3281     19.2520    147.4773     28.1115   
9537         218.2426    274.7572   2619.1004    674.3049    181.9868   
9769         147.4217    466.3677   4542.0178    614.1263    202.8739   
15239        257.8297    213.4336    876.4116   1554.1902    507.1274   
203          813.1225    864.8631   4407.3383   5406.7102    439.1032   

           GSM2667778  GSM2667779  GSM2667780  GSM2667781  GSM2667782  \
sample_id                                                               
27744        281.7885      1.5309     18.0041     14.8531    305.5563   
9537         715.1966     68.2822    985.3323    772.8154   1533.7694   
9769         289.3384    166.5569   1629.0443    566.4231   1570.2533   
15239       1001.3734     76.3392    493.3

In [32]:
train_df_mad = mad(train_df.drop(['gene', 'gene_symbol'], axis = 1))
train_df_mad.columns = ['gene_id', 'median_abs_deviation']

train_df_mad

ValueError: cannot reindex on an axis with duplicate labels

In [ ]:
# # Determine most variably expressed genes and subset
# train_df_mad = abs(train_df.drop(['gene', 'gene_symbol'], axis = 1) - train_df.drop(['gene', 'gene_symbol'], axis = 1).median()).median().reset_index()
# train_df_mad.columns = ['gene_id', 'median_abs_deviation']

# file = os.path.join('data', 'grein_mad_human_genes.tsv')
# train_df_mad.to_csv(file, sep='\t', index=False)